## การสร้าง Deep Learning API

ในหัวข้อนี้ เราจะเรียนวิธีการสร้าง Deep Learning REST API อย่างง่าย ด้วย Keras และ Flask web framework 

โดย REST API ที่เราจะสร้างในตัวอย่างนี้จะเป็น API สำหรับจำแนกรูปภาพของสุนัขพันธุ์ต่างๆ ดังรูปที่ 1.

![](api_arch.png)
<b><center>รูปที่ 1. REST API สำหรับจำแนกรูปภาพสุนัข</center></b>

*ตัวอย่างนี้ นำมาจาก: https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html

### Setup the Development Environment

#### Install Libraries

```
$ pip install flask gevent requests pillow
```

#### Designing Keras REST API

Keras REST API ของเราจะประกอบด้วย 3 ฟังก์ชัน ดังนี้ คือ
* load_the_model: เป็นฟังก์ชันสำหรับโหลด โมเดลการเรียนรู้เชิงลึก และเตรียมโมเดลให้พร้อมสำหรับการทำนาย
* prepare_image: เป็นฟังก์ชันสำหรับเตรียมรูปภาพให้พร้อมสำหรับป้อนเป็นอินพุทให้กับโมเดล
* predict: เป็น endpoint ของ API ของเรา ที่จะรับข้อมูลรูปภาพจาก client ทำนายชนิดของรูปภาพ และส่งผลการทำนายกลับไปให้กับ client

#### Implement the API

In [1]:
%%writefile run_keras_server.py
# import libraries
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image
import numpy as np
import flask
import io

# constants for image dimensions
IMG_WIDTH = 224
IMG_HEIGHT = 224

# initialize Flask app and the Keras model
app = flask.Flask(__name__)
model = None

def load_model():
    """Load the ResNet50 pre-trained model."""
    global model
    model = ResNet50(weights='imagenet')

def prepare_image(image, target):
    if image.mode != "RGB":
        image = image.convert("RGB")
    # resize and make the image ready as an input to ResNet50
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) 
    iamge = imagenet_utils.preprocess_input(image)
    
    return image

@app.route("/predict", methods=["POST"])
def predict():
    # initialize the data dictionary that will be returned from the view
    data = {"success" : False}
    
    if flask.request.method == "POST":
        if flask.request.files.get("image"):
            # read the image in PIL format
            image = flask.request.files["image"].read()
            image = Image.open(io.BytesIO(image))
            
            # preprocess the image
            image = prepare_image(image, target=(IMG_WIDTH, IMG_HEIGHT))
            
            # classify the image
            preds = model.predict(image)
            
            # format the prediction output
            results = imagenet_utils.decode_predictions(preds)
            data["predictions"] = []
            
            for (imagenetID, label, prob) in results[0]:
                r = {"label" : label, "probability": float(prob)}
                data["predictions"].append(r)
                
            data["success"] = True
            
    return flask.jsonify(data)

if __name__ == "__main__":
    print("* loading keras model and starting Flask server...")
    load_model()
    app.run()

Overwriting run_keras_server.py


#### Run the server

```
$ python run_keras_server.py
```

#### Makeing request using cURL

```
$ curl -X POST -F image=@dog.jpg 'http://localhost:5000/predict'
```

#### Making request programmatically

In [6]:
%%writefile simple_request.py
import requests

KERAS_REST_API_URL = 'http://localhost:5000/predict'
IMAGE_PATH = 'dog.jpg'

image = open(IMAGE_PATH, 'rb').read()
payload = {"image": image}

r = requests.post(KERAS_REST_API_URL, files=payload).json()

if r['success']:
    for (i, result) in enumerate(r['predictions']):
        print('{}. {}: {:.4f}'.format(i + 1, result['label'],
                                    resultb['probability']))
else:
    print('Request failed')

Overwriting simple_request.py


#### นำโมเดลที่สร้างขึ้นเองมาใช้แทน ResNet50 

In [3]:
%%writefile run_keras_server2.py
# import libraries
import keras.models
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image
import numpy as np
import flask
import io

# constants for image dimensions
IMG_WIDTH = 150
IMG_HEIGHT = 150

# initialize Flask app and the Keras model
app = flask.Flask(__name__)
model = None

def load_model():    
    """Load our cats and dogs VGG16 model."""    
    global model
    # Load our model 
    model = keras.models.load_model('cats_and_dogs-vgg16.h5')

def prepare_image(image, target):
    if image.mode != "RGB":
        image = image.convert("RGB")
    # resize and make the image ready as an input to ResNet50
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) 
    iamge = imagenet_utils.preprocess_input(image)
    
    return image

@app.route("/predict", methods=["POST"])
def predict():
    # initialize the data dictionary that will be returned from the view
    data = {"success" : False}
    
    if flask.request.method == "POST":
        if flask.request.files.get("image"):
            # read the image in PIL format
            image = flask.request.files["image"].read()
            image = Image.open(io.BytesIO(image))
            
            # preprocess the image
            image = prepare_image(image, target=(IMG_WIDTH, IMG_HEIGHT))
            
            # classify the image
            preds = model.predict(image)
            
            # build the results
            data["predictions"] = []            
            for pred in preds:
                if pred:
                    label = 'a dog'
                else:
                    label = 'a cat'
                r = {"label" : label, "probability": None}
                
                data["predictions"].append(r)
                
            data["success"] = True
            
    return flask.jsonify(data)

if __name__ == "__main__":
    print("* loading keras model and starting Flask server...")
    load_model()
    app.run()

Overwriting run_keras_server2.py


In [5]:
%%writefile simple_request2.py
import requests

KERAS_REST_API_URL = 'http://localhost:5000/predict'
IMAGE_PATH = 'dog.jpg'

image = open(IMAGE_PATH, 'rb').read()
payload = {"image": image}

r = requests.post(KERAS_REST_API_URL, files=payload).json()

if r['success']:
    for (i, result) in enumerate(r['predictions']):
        print('{}. {}'.format(i + 1, result['label']))
else:
    print('Request failed')

Overwriting simple_request2.py
